In [ ]:
from importnb import Notebook
with Notebook():
    import Acquisition_Traitement_Donnees  # Now Step1 is treated like a Python module
df=Acquisition_Traitement_Donnees.df

In [ ]:
import pandas as pd
# Ajout d'une colonne goal egale a 1 si goal 0 sinon
df['goal']=df['result'].apply(lambda x: 1 if x=='goal' else 0 )
df

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
def shottype(df,saison):
    df=df[df['season']==saison].copy()
    #Rassemble le nombre total de tir par type dans un dictionnaire
    df=df.groupby('shotType').agg(total_shots=('shotType','count'),goals=('goal',lambda x: (x==1).sum())).reset_index()
    s=str(saison)
    s=  s[:4] + '-' + s[4:]
    fig,ax = plt.subplots(figsize =(12, 8))
    # Position des bars sur l'axe X
    barWidth=0.3
    bar =  np.arange(len(df.index))
    bar2 = [x + barWidth for x in bar]
    colors = sns.color_palette("muted")
    df = df.sort_values(by='total_shots', ascending=False)
    # Creation des bars
    #total shots
    ax.bar(bar, df['total_shots'],color =colors[1], width = 0.25, 
            edgecolor ='grey', label ='Goals') 
    #goals
    ax.bar(bar2, df['goals'],color =colors[0], width = 0.25, 
            edgecolor ='grey', label ='Total shot')  

    # Labels
    plt.xlabel('Shot type', fontweight ='bold', fontsize = 15) 
    ax.set_ylabel('Number of shots ', fontweight ='bold', fontsize = 15) 
    #position des ticks et labels 
    plt.xticks([r+barWidth/2 for r in range(len(df.index))],df['shotType'].unique() )
    plt.title(f"Nombre de tirs et de buts par type de tir {s}",fontweight ='bold', fontsize = 15)
    plt.legend()
    plt.show() 
shottype(df,20182019)    

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns    

def stackedshottype(df, saison):
    df = df[df['season'] == saison].copy()
    df = df.groupby('shotType').agg(total_shots=('shotType', 'count'), goals=('goal', lambda x: (x == 1).sum())).reset_index()
    df['pourcentage_but'] = (df['goals'] / df['total_shots']) * 100
    s = str(saison)
    s = s[:4] + '-' + s[4:]
    df = df.sort_values(by='total_shots', ascending=False)

    # Creation de la figure
    fig, ax = plt.subplots(2, figsize=(12, 12))  # Increase height for better separation of subplots

    # Position des bars sur l'axe X
    br1 = np.arange(7)  # First 7 values for the first subplot
    br2 = np.arange(4)  # Remaining 4 values for the second subplot
    colors = sns.color_palette("muted")  

    # First subplot: Plot for first 7 shot types
    ax[0].bar(br1, df.goals.head(7), color=colors[0], width=0.5, edgecolor='grey', label='Goals')
    ax[0].bar(br1, (df.total_shots - df.goals).head(7), bottom=df.goals.head(7), color=colors[1], width=0.5, 
              edgecolor='grey', label='Total Shots')

    # Annotate first 7 bars with percentages
    for i,(goals,pct) in enumerate(zip(df.goals.head(7).values,df.pourcentage_but.head(7).values)):
        ax[0].annotate(f'{pct:.2f}%', (i, goals), xytext=(0, 10), textcoords='offset points', ha='center', va='top')

    # Second subplot: Plot for last 4 shot types
    ax[1].bar(br2, df.goals.tail(4), color=colors[0], width=0.5, edgecolor='grey', label='Goals')
    ax[1].bar(br2, (df.total_shots - df.goals).tail(4), bottom=df.goals.tail(4), color=colors[1], width=0.5, 
              edgecolor='grey', label='Total Shots')

    # Annotate last 4 bars with percentages  
    for i,(goals,pct) in enumerate(zip(df.goals.tail(4).values,df.pourcentage_but.tail(4).values)):
             ax[1].annotate(f'{pct:.2f}%', (i, goals), xytext=(0, 10), textcoords='offset points', ha='center', va='top')

    # Add labels and customize axes
    for ax_ in ax:
        ax_.set_ylabel('Number of shots ', fontweight='bold', fontsize=15)
    # Set xticks and title for both subplots
    ax[0].set_xticks(br1)
    ax[0].set_xticklabels(df.shotType.head(7).unique())
    ax[1].set_xticks(br2)
    ax[1].set_xticklabels(df.shotType.tail(4).unique())

    # Add titles and legends
    fig.suptitle(f"Nombre de tirs et de goals et pourcentage {s}", fontweight='bold', fontsize=15)
    ax[0].legend(loc='upper center')
    ax[1].legend(loc='upper center')
    
    plt.tight_layout()
    plt.show()

stackedshottype(df, 20222023)


Question 1a.	Quelle est la relation entre la distance à laquelle un tir a été effectué et la chance qu'il s'agisse d'un but? Produisez un graphique pour chaque saison entre 2018-19 et 2020-21 pour répondre à cette question, et ajoutez-le à votre article de blog avec quelques phrases décrivant le graphique. Y a-t-il eu beaucoup de changements au cours des trois dernières saisons? Pourquoi est-ce que vous avez choisi ce type de graphique?


In [ ]:
import numpy as np
"""def goaldistance(x):
    Retourne la distance euclidienne entre des coordonnees (x,y) et le but
        On assume les coordonnées du goal sont (89,0) et (-89,0), et on assume que les tirs du côté droit de la patinoire vont vers le but droit et vice versa.
    Args:
        x (pd.Series): (x,y)

    Returns:
        _type_: float

    if x[0] >= 0: #Verifie si le x est superieur a 0, base le tir sur ca 
        return np.linalg.norm(x - np.array([89, 0]))
    else:
        return np.linalg.norm(x - np.array([-89, 0]))"""
#On sépare la colonne coordinates en 2 colonnes x et y 
df['coordinates'] = df['coordinates'].str.replace(r'\s+', '', regex=True)  # Remove all whitespace
df[['x','y']] = df['coordinates'].str.extract(r'\(\s*([^,]+)\s*,\s*([^)]+)\s*\)').apply(pd.to_numeric, errors='coerce')
#On décide de drop les tirs derriere le but
df.drop(df[(df['x'] < -89)].index, inplace=True)
df.drop(df[(df['x'] >89)].index, inplace=True)
df = df[df['x'].notna() & df['y'].notna()]
#Il nya pas beaucoup de valeurs manquantes on decide de les drop

#Calcul de la distance euclidienne avec x et y
#df['distance'] = df[['x', 'y']].apply(lambda x: goaldistance(x.to_numpy()), axis=1)
# Vectorized calculation 
right_side = df['x'] >= 0
df['distance'] = np.where(
    right_side,
    np.sqrt((df['x'] - 89)**2 + (df['y'] - 0)**2),
    np.sqrt((df['x'] + 89)**2 + (df['y'] - 0)**2)
)

Choix du graphe : Line Plot

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import pandas as pd
bins=np.arange(0,110,10)
df['distance_bins'] = pd.cut(df['distance'], bins=bins,include_lowest=True,right=True)
goal_ratio = df.groupby(['season', 'distance_bins'])['goal'].mean().reset_index()
fig,ax = plt.subplots(figsize =(12, 8)) 
colors = sns.color_palette("muted")  
#IL partage l'index X et les labels de distance
xticks=goal_ratio[goal_ratio['season']==20182019]['goal'].index
label_distance=goal_ratio[goal_ratio['season']==20182019]['distance_bins']
ax.plot(xticks,goal_ratio[goal_ratio['season']==20182019]['goal'],color =colors[0], marker='o', linewidth=2, markersize=8, label='2018-2019') 
ax.plot(xticks,goal_ratio[goal_ratio['season']==20192020]['goal'],color =colors[1], marker='o', linewidth=2, markersize=8, label='2019-2020') 
ax.plot(xticks,goal_ratio[goal_ratio['season']==20202021]['goal'],color =colors[2], marker='o', linewidth=2, markersize=8, label='2020-2021') 

# Titre
ax.set_title('Taux de Buts en Fonction de la Distance au But par Saison (2018-2021)',fontweight ='bold', fontsize = 15) 
#Labels 
plt.xlabel('Distance du but', fontweight ='bold', fontsize = 15) 
plt.ylabel('Goal ratio', fontweight ='bold', fontsize = 15) 
plt.xticks(xticks, label_distance )
plt.legend()
plt.show() 
df

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import pandas as pd
def distance_ratio(df,saison):
    bins=np.arange(0,110,10)
    df=df[df['season']==saison].copy()
    s=str(saison)
    s=  s[:4] + '-' + s[4:]
    #On catégorise la distance en differents bins 
    df['distance_bins'] = pd.cut(df['distance'], bins=bins,include_lowest=True,right=True)
    #Calcul du ratio
    goal_ratio = df.groupby(['season', 'distance_bins'])['goal'].mean().reset_index()
    fig,ax = plt.subplots(figsize =(12, 8)) 
    colors = sns.color_palette("muted")  
    #IL partage l'index X et les labels de distance
    index_plot=goal_ratio.index
    label_distance=goal_ratio['distance_bins']
    ax.plot(index_plot,goal_ratio['goal'],color =colors[0], marker='o', linewidth=2, markersize=8, label=s)
    # Ticks
    ax.set_title(f'Taux de Buts en Fonction de la Distance au But par Saison ({s})',fontweight ='bold', fontsize = 15)  
    plt.xlabel('Distance du but', fontweight ='bold', fontsize = 15) 
    plt.ylabel('Goal ratio', fontweight ='bold', fontsize = 15) 
    plt.xticks(index_plot, label_distance )
    plt.legend()
    plt.show() 
distance_ratio(df,20182019)

In [ ]:
distance_ratio(df,20192020)

In [ ]:
distance_ratio(df,20202021)

Question 2.	Combinez les informations des sections précédentes pour produire un graphique qui montre le pourcentage de buts (# buts / # tirs) en fonction à la fois de la distance par rapport au filet et de la catégorie de types de tirs (vous pouvez choisir une seule saison de votre choix). Discutez brièvement de vos conclusions. Par exemple, quels sont les types de tirs les plus dangereux?
Choix du graphe : Heatmap

In [ ]:
import seaborn as sns
def distance_ratio_typetir(df,saison):
    heat=df[df['season']==saison].copy()
    s=str(saison)
    s=  s[:4] + '-' + s[4:]
    #Ratio des goals par rapport a la distance et au type de tir
    heat=heat.groupby(['distance_bins','shotType'])['goal'].mean().reset_index()
    #Pourcentage
    heat['goal']=heat['goal']*100
    #PIvot pour le heatmap
    df_heat=heat.pivot_table(values='goal',index='shotType',columns='distance_bins')
    df_heat.fillna(0,inplace=True)
    fig,ax=plt.subplots(figsize=(10,8))
    ax=sns.heatmap(df_heat,annot=True,linewidths=0.7,cmap='viridis',cbar_kws={'label': 'Taux de Probabilité de Marquer un But (%)'})
    ax.set_title(f"Pourcentage de Buts en Fonction de la Distance et des Types de Tirs ({(s)})",fontweight ='bold', fontsize = 15,pad=40)
    ax.set_xlabel("Distance du but",fontweight ='bold', fontsize = 15)
    ax.set_ylabel("Type de tir",fontweight ='bold', fontsize = 15)
distance_ratio_typetir(df,20222023)

In [11]:
#df.to_csv("../complexe.csv",index=False)